# LDAP

Did attempt to integrate flask-ldapconn with CherryPy but it's too integrated and checks the app is running.


In [21]:
import cherrypy, ssl
from ldap3 import Server, Connection, ALL, SAFE_SYNC, SYNC, AUTO_BIND_TLS_BEFORE_BIND, SIMPLE, SUBTREE

# Generic

ssl_defaults = ssl.get_default_verify_paths()

cherrypy.config.setdefault('LDAP_SERVER', 'localhost')
cherrypy.config.setdefault('LDAP_PORT', 389)
cherrypy.config.setdefault('LDAP_BINDDN', None)
cherrypy.config.setdefault('LDAP_SECRET', None)
cherrypy.config.setdefault('LDAP_CONNECT_TIMEOUT', 10)
cherrypy.config.setdefault('LDAP_READ_ONLY', False)
cherrypy.config.setdefault('LDAP_VALID_NAMES', None)
cherrypy.config.setdefault('LDAP_PRIVATE_KEY_PASSWORD', None)
cherrypy.config.setdefault('LDAP_RAISE_EXCEPTIONS', False)
cherrypy.config.setdefault('LDAP_CONNECTION_STRATEGY', SYNC)
cherrypy.config.setdefault('LDAP_USE_SSL', False)
cherrypy.config.setdefault('LDAP_USE_TLS', True)
cherrypy.config.setdefault('LDAP_TLS_VERSION', ssl.PROTOCOL_TLSv1)
cherrypy.config.setdefault('LDAP_REQUIRE_CERT', ssl.CERT_REQUIRED)
cherrypy.config.setdefault('LDAP_CLIENT_PRIVATE_KEY', None)
cherrypy.config.setdefault('LDAP_CLIENT_CERT', None)
cherrypy.config.setdefault('LDAP_CA_CERTS_FILE', ssl_defaults.cafile)
cherrypy.config.setdefault('LDAP_CA_CERTS_PATH', ssl_defaults.capath)
cherrypy.config.setdefault('LDAP_CA_CERTS_DATA', None)
cherrypy.config.setdefault('FORCE_ATTRIBUTE_VALUE_AS_LIST', False)



# Overrides

cherrypy.config.update({
'LDAP_SERVER': 'localhost',
'LDAP_PORT': 10389,
'LDAP_BINDDN': 'cn=admin,dc=planetexpress,dc=com',
'LDAP_SECRET': 'GoodNewsEveryone',
'LDAP_BASEDN': 'dc=planetexpress,dc=com',
'LDAP_SEARCH_ATTR': 'mail',
#'LDAP_SEARCH_FILTER': '(mail=%s)' % USER_EMAIL,
#'LDAP_QUERY_FILTER': 'email: %s' % USER_EMAIL,
'LDAP_TLS_VERSION': ssl.PROTOCOL_TLSv1,
'LDAP_REQUIRE_CERT': ssl.CERT_NONE,
'LDAP_AUTH_BASEDN': 'ou=people,dc=planetexpress,dc=com',
'LDAP_AUTH_ATTR': 'mail',
'LDAP_AUTH_SEARCH_FILTER': '(objectClass=inetOrgPerson)'})

## Binding directly with ldap

In [8]:
from ldap3 import Server, Connection, ALL, SAFE_SYNC, SYNC, AUTO_BIND_TLS_BEFORE_BIND, SIMPLE

In [9]:
server = Server(host=cherrypy.config['LDAP_SERVER'],
            port=cherrypy.config['LDAP_PORT'],
            use_ssl=cherrypy.config['LDAP_USE_SSL'],
            connect_timeout=cherrypy.config['LDAP_CONNECT_TIMEOUT'],
            get_info=ALL)

In [10]:
username = 'Hubert J. Farnsworth'
password = 'professor'

uname = f'cn={username},{cherrypy.config["LDAP_AUTH_BASEDN"]}'
uname

'cn=Hubert J. Farnsworth,ou=people,dc=planetexpress,dc=com'

In [11]:
conn = Connection(server, uname, password, client_strategy=SAFE_SYNC)

In [12]:
conn.bind()

(True,
 {'result': 0,
  'description': 'success',
  'dn': '',
  'message': '',
  'referrals': None,
  'saslCreds': None,
  'type': 'bindResponse'},
 None,
 None)

In [13]:
status, result, response, _ = conn.bind()
status, result, response, _

(True,
 {'result': 0,
  'description': 'success',
  'dn': '',
  'message': '',
  'referrals': None,
  'saslCreds': None,
  'type': 'bindResponse'},
 None,
 None)

In [14]:
conn.unbind()

(True,
 {'result': 0,
  'description': 'success',
  'dn': '',
  'message': '',
  'referrals': None,
  'saslCreds': None,
  'type': 'bindResponse'},
 None,
 None)

In [15]:
admin_conn = Connection(server, 
                    cherrypy.config['LDAP_BINDDN'],
                    cherrypy.config['LDAP_SECRET'],
                    client_strategy=SAFE_SYNC)

In [16]:
cherrypy.config.setdefault('LDAP_RAISE_EXCEPTIONS', False)

cherrypy.config.setdefault('LDAP_CONNECTION_STRATEGY', SYNC)

auto_bind_strategy = AUTO_BIND_TLS_BEFORE_BIND
authentication_policy = SIMPLE
if cherrypy.config['LDAP_USE_TLS'] is not True:
    auto_bind_strategy = AUTO_BIND_NO_TLS

ldap_conn = Connection(
    server,
    auto_bind=auto_bind_strategy,
    client_strategy=cherrypy.config['LDAP_CONNECTION_STRATEGY'],
    raise_exceptions=cherrypy.config['LDAP_RAISE_EXCEPTIONS'],
    authentication=authentication_policy,
    user=cherrypy.config['LDAP_BINDDN'],
    password=cherrypy.config['LDAP_SECRET'],
    check_names=True,
    read_only=cherrypy.config['LDAP_READ_ONLY'],
)

In [17]:
ldap_conn.bind()

True

In [18]:
ldap_conn.search('mail=fry@planetexpress.com', cherrypy.config['LDAP_AUTH_SEARCH_FILTER'])  # usually you don't need the original request (4th element of the returned tuple)

False

In [22]:
search_filter = '(mail=fry@planetexpress.com)'
attr = cherrypy.config['LDAP_SEARCH_ATTR']
ldap_conn.search(cherrypy.config['LDAP_BASEDN'],
                 search_filter,
                 SUBTREE, 
                 attributes=[attr])

True

In [23]:
result = ldap_conn.result
response = ldap_conn.response

In [24]:
result

{'result': 0,
 'description': 'success',
 'dn': '',
 'message': '',
 'referrals': None,
 'type': 'searchResDone'}

In [25]:
response

[{'raw_dn': b'cn=Philip J. Fry,ou=people,dc=planetexpress,dc=com',
  'dn': 'cn=Philip J. Fry,ou=people,dc=planetexpress,dc=com',
  'raw_attributes': {'mail': [b'fry@planetexpress.com']},
  'attributes': {'mail': ['fry@planetexpress.com']},
  'type': 'searchResEntry'}]

# Phase 2 - LDAPS

In [26]:
import cherrypy, ssl
from ldap3 import Server, Connection, ALL, SAFE_SYNC, SYNC, AUTO_BIND_TLS_BEFORE_BIND, SIMPLE

# Generic

ssl_defaults = ssl.get_default_verify_paths()

cherrypy.config.setdefault('LDAP_SERVER', 'localhost')
cherrypy.config.setdefault('LDAP_PORT', 389)
cherrypy.config.setdefault('LDAP_BINDDN', None)
cherrypy.config.setdefault('LDAP_SECRET', None)
cherrypy.config.setdefault('LDAP_CONNECT_TIMEOUT', 10)
cherrypy.config.setdefault('LDAP_READ_ONLY', False)
cherrypy.config.setdefault('LDAP_VALID_NAMES', None)
cherrypy.config.setdefault('LDAP_PRIVATE_KEY_PASSWORD', None)
cherrypy.config.setdefault('LDAP_RAISE_EXCEPTIONS', False)
cherrypy.config.setdefault('LDAP_CONNECTION_STRATEGY', SYNC)
cherrypy.config.setdefault('LDAP_USE_SSL', False)
cherrypy.config.setdefault('LDAP_USE_TLS', True)
cherrypy.config.setdefault('LDAP_TLS_VERSION', ssl.PROTOCOL_TLSv1)
cherrypy.config.setdefault('LDAP_REQUIRE_CERT', ssl.CERT_REQUIRED)
cherrypy.config.setdefault('LDAP_CLIENT_PRIVATE_KEY', None)
cherrypy.config.setdefault('LDAP_CLIENT_CERT', None)
cherrypy.config.setdefault('LDAP_CA_CERTS_FILE', ssl_defaults.cafile)
cherrypy.config.setdefault('LDAP_CA_CERTS_PATH', ssl_defaults.capath)
cherrypy.config.setdefault('LDAP_CA_CERTS_DATA', None)
cherrypy.config.setdefault('FORCE_ATTRIBUTE_VALUE_AS_LIST', False)



# Overrides

cherrypy.config.update({
'LDAP_SERVER': 'localhost',
'LDAP_PORT': 10636,
'LDAP_BINDDN': 'cn=admin,dc=planetexpress,dc=com',
'LDAP_SECRET': 'GoodNewsEveryone',
'LDAP_BASEDN': 'dc=planetexpress,dc=com',
'LDAP_SEARCH_ATTR': 'mail',
'LDAP_TLS_VERSION': ssl.PROTOCOL_TLSv1_2,
'LDAP_REQUIRE_CERT': ssl.CERT_NONE,
'LDAP_AUTH_BASEDN': 'ou=people,dc=planetexpress,dc=com',
'LDAP_AUTH_ATTR': 'mail',
'LDAP_USE_SSL': True,
'LDAP_AUTH_SEARCH_FILTER': '(objectClass=inetOrgPerson)'})

In [27]:
ldaps_server = Server(host=cherrypy.config['LDAP_SERVER'],
            port=cherrypy.config['LDAP_PORT'],
            use_ssl=cherrypy.config['LDAP_USE_SSL'],
            connect_timeout=cherrypy.config['LDAP_CONNECT_TIMEOUT'],
            get_info=ALL)

In [28]:
username = 'Hubert J. Farnsworth'
password = 'professor'

uname = f'cn={username},{cherrypy.config["LDAP_AUTH_BASEDN"]}'
uname

'cn=Hubert J. Farnsworth,ou=people,dc=planetexpress,dc=com'

In [29]:
ldaps_server

Server(host='localhost', port=10636, use_ssl=True, allowed_referral_hosts=[('*', True)], tls=Tls(validate=<VerifyMode.CERT_NONE: 0>), get_info='ALL', connect_timeout=10, mode='IP_V6_PREFERRED')

In [30]:

conn = Connection(ldaps_server, uname, password, client_strategy=SAFE_SYNC)

In [31]:
conn.bind()

(True,
 {'result': 0,
  'description': 'success',
  'dn': '',
  'message': '',
  'referrals': None,
  'saslCreds': None,
  'type': 'bindResponse'},
 None,
 None)

In [32]:
conn.unbind()

(True,
 {'result': 0,
  'description': 'success',
  'dn': '',
  'message': '',
  'referrals': None,
  'saslCreds': None,
  'type': 'bindResponse'},
 None,
 None)